# doi_demo.ipynb
### demo reading in and making visualizations from camera/imu data as .nc files, .json files of ephys data, .npy files of ephys properties
Feb 12, 2021

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.insert(0, r'C:/Users/Angie Michaiel/Documents/GitHub/FreelyMovingEphys')
from util.ephys import ephys_to_dataset
from util.paths import find

In [3]:
recordings_path = r'T:\freely_moving_ephys\ephys_recordings'
dates_path = ['020821']

In [4]:
collected_ephys = ephys_to_dataset(recordings_path, dates_path)

UnboundLocalError: local variable 'all_units_xr' referenced before assignment

In [ ]:
collected_ephys

## ephys props

In [ ]:
for unit,

In [ ]:
# visualize one unit's STA
plt.imshow(collected_ephys.EE12P1RN_020821_control_Rig2_hf1_wn_unit9.sel(ephys_params='sta').values.ravel()[0],vmin=-0.3,vmax=0.3,cmap = 'jet')
plt.colorbar()

In [ ]:
plt.imshow(collected_ephys.EE12P1RN_020821_control_Rig2_fm1_unit9.sel(ephys_params='sta').values.ravel()[0],vmin=-0.3,vmax=0.3,cmap = 'jet')
plt.colorbar()

In [1]:
# visualize the STA of units meeting a given criteria
for unit, data in collected_ephys.filter_by_attrs(unit='unit9').data_vars.items():
    plt.figure()
    plt.title(unit); plt.imshow(data.sel(ephys_params='sta').values.ravel()[0],vmin=-0.3,vmax=0.3,cmap = 'jet')
    plt.show()

NameError: name 'collected_ephys' is not defined

In [ ]:
for unit, data in collected_ephys.filter_by_attrs(unit='unit9').data_vars.items():
    plt.figure()
    plt.title(unit); plt.plot(data.sel(ephys_params='contrast_range').values.ravel()[0], data.sel(ephys_params='contrast_response').values.ravel()[0])
    plt.show()

## combining ephys analysis with the arena and mouse-mounted cameras

In [ ]:
top_files = find('020821*fm1_TOP1.nc', recordings_path)
topdown_dataset = xr.open_dataset(top_files[0])

In [ ]:
topdown_data = topdown_dataset.TOP1_pts

what are the best points for the topdown view?

In [ ]:
topdown_pt_names = list(topdown_data['point_loc'].values)
likeli_loop_count = 0
for pt_num in range(0, len(topdown_pt_names)):
    current_pt_loc = topdown_pt_names[pt_num]
    if 'likelihood' in current_pt_loc:
        # find the associated x and y points of the selected likelihood
        # assumes order is x, y, likelihood, will cause problems if isn't true of data...
        assoc_x_pos = topdown_pt_names[pt_num - 2]
        assoc_x_pt = topdown_data.sel(point_loc=assoc_x_pos)
        assoc_y_pos = topdown_pt_names[pt_num - 1]
        assoc_y_pt = topdown_data.sel(point_loc=assoc_y_pos)
        likeli_pt = topdown_data.sel(point_loc=current_pt_loc).values
        assoc_x_pt[likeli_pt < 0.99] = np.nan
        assoc_y_pt[likeli_pt < 0.99] = np.nan
        x_perc_good = np.sum(~np.isnan(assoc_x_pt))/len(assoc_x_pt)
        y_perc_good = np.sum(~np.isnan(assoc_y_pt))/len(assoc_y_pt)
        print(current_pt_loc, 'x', round(float(x_perc_good*100),3), 'y', round(float(y_perc_good*100),3))

best point was at the base of the tail

In [ ]:
x_trace = top_dataset.TOP1_pts.sel(point_loc='tailbase_x').values
y_trace = top_dataset.TOP1_pts.sel(point_loc='tailbase_y').values
plt.plot(x_trace, y_trace,'k.',alpha=0.2)

In [ ]:
# load in the json that stores all of the spike times from ephys recording
ephys_json_path = find('020821*fm1*.json', recordings_path)
ephys_data = pd.read_json(ephys_json_path[0])
goodcells = ephys_data.loc[ephys_data['group']=='good']

In [ ]:
# get the spike times of each unit, and plot where spikes happen in the arena
# also see /project_analysis/general/place_cells.ipynb
units = goodcells.index.values
for i, ind in enumerate(goodcells.index):
    unit_spikes = np.array(goodcells.at[units[i],'spikeT']) # spike times in seconds
    spike_positions = []
    for s in unit_spikes:
        pos_at_spike = [x_trace[int(s*60)], y_trace[int(s*60)]]
        spike_positions.append(pos_at_spike)
    spike_positions = np.array(spike_positions)
    plt.figure()
    plt.title('unit'+str(ind))
    plt.plot(x_trace, y_trace, 'k.', alpha=0.2)
    plt.plot(spike_positions[:,0], spike_positions[:,1], 'r.')

In [ ]:
# comparing eye movements in DOI vs noDOI recordings
reye_files = find('020821*Reye.nc', recordings_path)
reye_dataset = xr.open_dataset(reye_files[0])
reye_dataset

In [ ]:
pupil_radius = reye_dataset.REYE_ellipse_params.sel(ellipse_params='longaxis').values
plt.plot(pupil_radius)
plt.xlim(5000,7000)

In [ ]:
# comparing head movements in DOI vs noDOI recordings
imu_dataset_path = find('020821*fm1_imu.nc', recordings_path)
imu_dataset = xr.open_dataset(imu_dataset_path[0])

In [ ]:
gyro_z = imu_dataset.IMU_data.sel(sample='gyro_z').values
plt.plot(gyro_z)
plt.xlim(5000,7000)